<a href="https://colab.research.google.com/github/kkimtaejung/All_heuristic/blob/GAN/ROBO_GAN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN 사전지식

## 백쿼리

### 백쿼리 (역질의)
- 기존의 MNIST는 (28*28) -> 784 -> 10개의 레이블로 출력
- 10개의 레이블 출력값 -> 784 0=-> (28*28) 로 데이터 생성 -> 백쿼리!

### 백쿼리 특징
- 같은 원핫 인코딩 벡터 -> 같은 결과 출력
- 특정 레이블을 나타내는 모든 훈련 데이터의 평균적인 이미지가 나온다

### 백쿼리 한계
- 평균같은 애매모호한 이미지가 아니라 훈련 샘플같은 이미지를 원함
- 각각 서로 다른 이미지들을 만들어내길 원함
- 간단한 백쿼리로는 불가능

## 적대적 훈련

1. 구조적으로 새로운 신경망

 고양이, 고양이가 아님 -> 분류기에 넣고 -> 1 또는 0을 출력
2. 고양이가 아닌 이미지를 자동으로 생성하는 생성기 구조
 
 진짜 고양이, 가짜 고양이 생성기 -> 분류기에 넣고 -> 1또는 0을 출력
3. 생성기에 보상과 벌을 주는 구조

 진짜 그림, 생성기 -> 판별기에 넣고 -> 1또는 0을 출력

 1) 생성기가 판별기를 속이면 보상을 줌
 
 2) 생성기가 판별기에 잡히면 벌을 줌

4. 생성기, 판별기를 동시에 훈련 (동시에 한다는게 중요!!!)

 생성기에서 진짜 그림과 분간이 어려운 그림을 뽑아낼 수 있겠다!
 
 생성적 적대 신경망 = GAN

## GAN 훈련 방법

1단계)

- 판별기에 실제 데이터를 보여주고 1.0이라는 값이어야한다고 알려줌
- 오차가 발생하면 이를 판별기를 업데이트하는데 사용함

2단계)

- 판별기에 생성기로 만들어진 가짜 데이터를 보여주고 0.0이어야 한다고 알려줌
- 오차는 판별기만을 업데이트하는데 사용, 이 단계까지 생성기는 업데이트하지 못하도록 해야함

3단계)

- 판별기에 생성기의 결과를 보여주고, 생성기에 결과가 1.0이어야 한다고 알려줌
- 오차는 생성기만을 업데이트하는데 사용, 이 단계에서 판별기는 업데이트 하지 않는다

### GAN 훈련의 어려움

판별기와 생성기의 훈련이 균형을 맞아야함

판별기가 너무 빨리 훈련되면 생성기가 못따라감

생성기가 너무 빨리 훈련되면 생성기가 조악한 이미지로 점수를 계속 획득하게 됨

## 단순한 1010 패턴

구조 : 실제 데이터 소스 [1 0 1 0] , 생성기 -> 판별기를 거쳐 -> 1또는 0의 값을 출력

GPU 사용

In [ ]:
# import torch
# import torchvision
# if torch.cuda.is_available():
#   torch.set_default_tensor_type(torch.cuda.FloatTensor)
#   print("using cuda: ", torch.cuda.get_device_name(0))
#   pass
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device

#### 실제 데이터 소스

In [ ]:
import torch
import torch.nn as nn
import pandas
import matplotlib.pyplot as plt
import random

# 실제 데이터 소스 (이상적인 값)
def generate_real0():
  real_data = torch.FloatTensor([1,0,1,0])
  return real_data
# 실제 데이터 소스 (현실적인 값)
def generate_real():
  real_data = torch.FloatTensor(
      [random.uniform(0.8,1.0),
       random.uniform(0.0,0.2),
       random.uniform(0.8,1.0),
       random.uniform(0.0,0.2)])
  return real_data
# 임의의 노이즈 패턴을 반환하는 함수 -> generate_real과 비슷하지만, 더 일반적이고, 텐서 크기를 매개변수로 가지는 유연한 함수임
  # 즉, 생성기에서 임의 값 뽑아내는 것 같은 역할
def generate_random(size):
  random_data = torch.rand(size)
  return random_data

#### 판별기 만들기

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(4,3),
        nn.Sigmoid(),
        nn.Linear(3,1),
        nn.Sigmoid())
    self.loss_function = nn.MSELoss()
    self.optimiser = torch.optim.SGD(self.parameters(), lr = 0.01)
    self.counter = 0

  def forward(self, inputs):
    return self.model(inputs)

  def train(self,inputs,targets):
    outputs = self.forward(inputs)
    loss = self.loss_function(outputs,targets)
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
    if(self.counter%10000==0):
      print("counter = ",self.counter)
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(4,3),
        nn.Sigmoid(),
        nn.Linear(3,1),
        nn.Sigmoid())
    # 구조 :: node4 -> node3(로지스틱) -> node1(로지스틱) -> 1 or 0

    # 손실함수 설정
    self.loss_function = nn.MSELoss()

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.SGD(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,targets):
    #신경망 출력 계산
    outputs = self.forward(inputs)

    #손실 계산
    loss = self.loss_function(outputs,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
  
  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

#### 판별기 훈련 반복



In [ ]:
D = Discriminator()
for i in range(10000):
  #실제 데이터
  D.train(generate_real(), torch.FloatTensor([1.0]))
  #생성된 데이터
  D.train(generate_random(4), torch.FloatTensor([0.0]))

In [ ]:
# 손실함수 차트
D.plot_progress()

In [ ]:
# 판별기 테스트 (패턴 넣어보기)
print(D.forward(generate_real()).item())
print(D.forward(generate_random(4)).item())

여기까지가 판별기에 데이터를 넣었을 때, 진짜 가짜를 구별하는 정도

#### 생성기 만들기

::구조::

node1 -> node3(로지스틱) -> node4(로지스틱)

판별기와 정반대의 구조인 이유는, 판별기와 생성기를 균형있게 훈련시키기 위함이다.

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(1,3),
        nn.Sigmoid(),
        nn.Linear(3,4),
        nn.Sigmoid())
    self.optimiser = torch.optim.SGD(self.parameters(), lr = 0.01)
    self.counter = 0

  def forward(self, inputs):
    return self.model(inputs)

  def train(self,D,inputs,targets):
    g_output = self.forward(inputs)
    d_output = D.forward(g_output)
    loss = D.loss_function(d_output,targets)
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(1,3),
        nn.Sigmoid(),
        nn.Linear(3,4),
        nn.Sigmoid())
    # 구조 :: node1 -> node3(로지스틱) -> node4(로지스틱)

    # 손실함수 없음 -> 오직 판별기에만 필요
    # 판별기의 흘러들어온 기울기 오차 -> 생성기 업데이트

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.SGD(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,D,inputs,targets):
    #신경망 출력 계산
    g_output = self.forward(inputs)

    #판별기로 전달
    d_output = D.forward(g_output)

    #오차 계산
    loss = D.loss_function(d_output,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
# ::train::
# 입력값이 생성기의 forward로 전달됨
# 생성기의 출력 g_output은 D.forward를 통해 판별기로 전달되어 d_output을 출력함
# 손실 = |d_output - 정답| -> 차이값으로 계산
# 손실로부터 오류역전파 되어 생성기로 전달되어 생성기의 optimiser로 전달됨
# 이러면 생성기만 업데이트됨

  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

#### 생성기 결과 확인

In [ ]:
# 훈련되어있지 않기 때문에 임의의 4개의 값을 출력할거다.
G = Generator()
G.forward(torch.FloatTensor([0.5]))

#### GAN 훈련

In [ ]:
D = Discriminator()
G = Generator()
image_list=[]

for i in range(10000):
  # 1단계 : 참에 대해 판별기 훈련
  D.train(generate_real(), torch.FloatTensor([1.0]))
  # 2단계 : 거짓에 대해 판별기 훈련, 
  #         생성기의 기울기가 수정되지 않도록 detach()함수 이용, 
  #         detach()로 선택된 것은 그 함수에서 제외됨
  D.train(G.forward(torch.FloatTensor([0.5])).detach(), torch.FloatTensor([0.0]))
  # 3단계 : 생성기 훈련
  G.train(D, torch.FloatTensor([0.5]), torch.FloatTensor([1.0]))

  if(i%1000==0):
    image_list.append(G.forward(torch.FloatTensor([0.5])).detach().numpy())


In [ ]:
# 판별기, 생성기 생성
D = Discriminator()
G = Generator()
image_list=[]

# 판별기, 생성기 훈련
for i in range(10000):
  # 1단계 : 참에 대해 판별기 훈련
  D.train(generate_real(), torch.FloatTensor([1.0]))

  # 2단계 : 거짓에 대해 판별기 훈련, 생성기의 기울기가 수정되지 않도록 detach()함수 이용, detach()로 선택된 것은 그 함수에서 제외됨
  D.train(G.forward(torch.FloatTensor([0.5])).detach(), torch.FloatTensor([0.0]))

  # 3단계 : 생성기 훈련
  G.train(D, torch.FloatTensor([0.5]), torch.FloatTensor([1.0]))

  # 매 1000회 생성기 훈련 결과를 이미지로 저장
  if(i%1000==0):
    image_list.append(G.forward(torch.FloatTensor([0.5])).detach().numpy())

  pass

# ::detach::
# 입력 데이터가 판별기를 지나 출력되고, 다시 업데이트 되어 판별기로 돌아올때, 생성기로 가지 않게 해줌
# 효율적이고 빠른 계산을 위해 사용함

In [ ]:
D.plot_progress()

그래프를 보면 손실이 0.25 를 머뭄

의미는 판별기가 실제, 조작된 데이터를 잘 판별하지 못하는 결과를 내놓으면, 결과로 0.0이나 1.0 같은 확실한 결과가 아닌, 0.5라는 결과를 내놓은 것이고 평균제곱오차로 인해 0.25라는 손실이 나옴

In [ ]:
G.plot_progress()

In [ ]:
# 생성기가 만들어낸 데이터 패턴!
G.forward(torch.FloatTensor([0.5]))

In [ ]:
import numpy
plt.figure(figsize=(16,8))
plt.imshow(numpy.array(image_list).T, interpolation = 'none', cmap='Blues')

## 손으로 쓴 숫자 훈련 GAN

### 모델 구조

입력이 MNIST 이미지, 생성기 -> 판별기 -> 1 or 0 출력

### import & dataset

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas, numpy, random
import matplotlib.pyplot as plt

# 구글 드라이브 연결
# 파일 열어서 3번째 드라이브 마운트 누르면 됨

from torch.utils.data import Dataset
# Dataset 클래승서 데이터셋 상속받기 위해 2개의 특수 메서드 구현 필요
# __len__() : 데이터셋 길이 반환
# __getitem__() : 데이터셋의 n번째 아이템 반환
class MnistDataset(Dataset):
  def __init__(self,csv_file):
    self.data_df = pandas.read_csv(csv_file, header=None)
    pass

  def __len__(self):
    return len(self.data_df)

  def __getitem__(self,index):
    #이미지 레이블
    label = self.data_df.iloc[index,0]
    target = torch.zeros((10))
    target[label] = 1.0
    #1의 경우 [0,1,0,0,0,0,0,0,0,0] -> 원핫 인코딩 됨

    #normalization
    image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values)/255.0

    #레이블, 이미지데이터 텐서, 레이블 텐서 반환
    return label, image_values, target

  def plot_image(self, index):
    img = self.data_df.iloc[index,1:].values.reshape(28,28)
    plt.title("label = "+str(self.data_df.iloc[index,0]))
    plt.imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

# 데이터셋 불러오기
mnist_dataset = MnistDataset('/content/drive/MyDrive/ROBOTICS/GAN/mnist_train.csv')
mnist_dataset.plot_image(17)

### MNIST 판별기

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(784,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,1),
        nn.Sigmoid())
    # 구조 :: node4 -> node3(로지스틱) -> node1(로지스틱) -> 1 or 0

    # 손실함수 설정
    self.loss_function = nn.BCELoss()

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,targets):
    #신경망 출력 계산
    outputs = self.forward(inputs)

    #손실 계산
    loss = self.loss_function(outputs,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
  
  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

110pg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 판별기 테스트

In [ ]:
# 60000개의 임지에서 판별기가 진짜 숫자라고 판단하면 결과를 1.0으로 출력하도록 함
D=Discriminator()
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data
def generate_random_seed(size):
  random_data = torch.randn(size)
  return random_data
# for label, image_data_tensor, target_tensor in mnist_dataset:
#   # 실제 데이터
#   D.train(image_data_tensor, torch.FloatTensor([1.0]))
#   # 생성된 데이터
#   D.train(generate_random_image(784), torch.FloatTensor([0.0]))
#   pass

# D.plot_progress()

In [ ]:
# 실제 이미지, 임의의 노이즈 에 대해 판별기가 잘 구별했는지 확인
for i in range(5):
  image_data_tensor = mnist_dataset[random.randint(0,60000)][1]
  print(D.forward(image_data_tensor).item())
  pass

for i in range(5):
  print(D.forward(generate_random(784)).item())
  pass

### MNIST 생성기

::전체 구조::

임의의 시드 -> 생성기 1 -> 200 -> 784 -> 28*28 -> 784 -> 200 -> 1 판별기

::생성기의 입력은 random seed::

생성기는 다양한 이미지를 생성해야함 0~9까지

같은 입력에 대해서는 신경망은 같은 출력을 내기 때문에 입력을 임의의 값으로 항상 넣어주어야 한다.

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(100,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,784),
        nn.Sigmoid())
    # 구조 :: node1 -> node3(로지스틱) -> node4(로지스틱)

    # 손실함수 없음 -> 오직 판별기에만 필요
    # 판별기의 흘러들어온 기울기 오차 -> 생성기 업데이트

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,D,inputs,targets):
    #신경망 출력 계산
    g_output = self.forward(inputs)

    #판별기로 전달
    d_output = D.forward(g_output)

    #오차 계산
    loss = D.loss_function(d_output,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
# ::train::
# 입력값이 생성기의 forward로 전달됨
# 생성기의 출력 g_output은 D.forward를 통해 판별기로 전달되어 d_output을 출력함
# 손실 = |d_output - 정답| -> 차이값으로 계산
# 손실로부터 오류역전파 되어 생성기로 전달되어 생성기의 optimiser로 전달됨
# 이러면 생성기만 업데이트됨

  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

### 생성기 결과 확인

In [ ]:
# 새로운 생성기를 만들고, 임의의 시드로 출력 텐서를 만듬
G = Generator()
output = G.forward(generate_random_seed(100))
img = output.detach().numpy().reshape(28,28)
plt.imshow(img, interpolation='none', cmap='Blues')

아직 훈련이 안된 생성기는 임의의 패턴을 추출함

### GAN 훈련하기 MNIST

In [ ]:
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data
def generate_random_seed(size):
  random_data = torch.randn(size)
  return random_data

In [ ]:
# 판별기, 생성기 생성
D = Discriminator()
G = Generator()
image_list=[]

# 판별기, 생성기 훈련
for i in range(1):
  for label, image_data_tensor, target_tensor in mnist_dataset:
    # 1단계 : 참에 대해 판별기 훈련
    D.train(image_data_tensor, torch.FloatTensor([1.0]))

    # 2단계 : 거짓에 대해 판별기 훈련, 생성기의 기울기가 수정되지 않도록 detach()함수 이용, detach()로 선택된 것은 그 함수에서 제외됨
    D.train(G.forward(generate_random_seed(100)).detach(), torch.FloatTensor([0.0]))

    # 3단계 : 생성기 훈련
    G.train(D, generate_random_seed(100), torch.FloatTensor([1.0]))

    

# ::detach::
# 입력 데이터가 판별기를 지나 출력되고, 다시 업데이트 되어 판별기로 돌아올때, 생성기로 가지 않게 해줌
# 효율적이고 빠른 계산을 위해 사용함

### 결과 확인

In [ ]:
def plot_progress(self):
  df = pandas.DataFrame(self.progress, columns=['loss'])
  df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.',grid=True,yticks=(0,0.25,0.5,1.0,5.0))

D.plot_progress()

0.25에서 균형이 맞기 시작했지만,

이후 판별기가 앞서거나, 손실값이 다시 낮은 상태로 머물게 됨

In [ ]:
G.plot_progress()

손실함수 값이 증가함을 통해,

판별기 성능이 생성기보다 더 나은 구간이라고 볼 수 있음

In [ ]:
# 생성기에서 만들어낸 이미지 확인
# 2*3, 6개 이미지 출력
f, axarr = plt.subplots(2,3,figsize=(16,8))
for i in range(2):
  for j in range(3):
    output = G.forward(generate_random_seed(100))
    img = output.detach().numpy().reshape(28,28)
    axarr[i,j].imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

완전하지는 않지만 어느정도의 형태를 가지고 있다!

## 모드 붕괴

### 의미

생성기가 10개의 숫자를 다양하게 생성하지 못하고, 오직 하나만 만들거나 일부만 만들게 되는 것

### 이유

아직 밝혀진 바가 없다...

### GAN 훈련 성능 향상

- 손실함수 MSE -> BCE 로 바꾸기
- 활성화함수 LeakyReLU 사용하기
- 정규화 LayerNorm 하기
- 경사하강법은 Adam으로 하기
- 입력 시드에 1개의 값이 아닌 여러 값을 넣기 (100개정도)
- 입력의 랜더 시드의 코드를 변경, torch.rand -> torch.randn

## 시드로 실험하기

- 임의의 두 개의 시드 seed1, seed2 가 있다고 가정
- seed1, seed2, seed1과 seed2의 중간 값의 시드로 생성된 이미지는 어떨까?

In [ ]:
# seed 1
seed1 = generate_random_seed(100)
out1 = G.forward(seed1)
img1 = out1.detach().numpy().reshape(28,28)
plt.imshow(img1,interpolation='none',cmap='Blues')

In [ ]:
# seed 2
seed2 = generate_random_seed(100)
out2 = G.forward(seed2)
img2 = out2.detach().numpy().reshape(28,28)
plt.imshow(img2,interpolation='none',cmap='Blues')

In [ ]:
# seed1 ~ seed2 사이 일정한 간격으로 12개 시드 구하기
# seed1 -> seed2 로 변해가는 과정을 볼 수 있다
count = 0
f, axarr = plt.subplots(3,4,figsize=(16,8))
for i in range(3):
  for j in range(4):
    seed = seed1+(seed2-seed1)/11*count
    output = G.forward(seed)
    img = output.detach().numpy().reshape(28,28)
    axarr[i,j].imshow(img, interpolation='none', cmap='Blues')
    count = count + 1
    pass
  pass

In [ ]:
# seed1 + seed2 한 시드 구하기
seed3 = seed1 + seed2
out3 = G.forward(seed3)
img3 = out3.detach().numpy().reshape(28,28)
plt.imshow(img3,interpolation='none',cmap='Blues')

In [ ]:
# seed1 - seed2 한 시드 구하기
seed4 = seed1 - seed2
out4 = G.forward(seed4)
img4 = out4.detach().numpy().reshape(28,28)
plt.imshow(img4,interpolation='none',cmap='Blues')

시드를 더하고 빼는건 우리가 생각하는 픽셀값을 더하고 빼는것과 다르다!

# GPU 사용

In [ ]:
import torch
import torchvision
import torch.nn
!pip3 install --upgrade torch torchvision

In [ ]:
torch.cuda.is_available()